In [79]:
import pandas as pd
import numpy as np

In [80]:
data = pd.read_csv('Data_preprocessed/Test.csv', index_col=0)

In [81]:
data.head()

,sentence,sentiment,topic,sentence_dropped
0,nói tiếng anh lưu loát .,2,0,nói tiếng lưu_loát
1,giáo viên rất vui tính .,2,0,giáo_viên vui_tính
2,cô max có tâm .,2,0,cô max tâm
3,"giảng bài thu hút , dí dỏm .",2,0,giảng thu_hút
4,"giáo viên không giảng dạy kiến thức , hướng dẫ...",0,0,giáo_viên NOT giảng_dạy kiến_thức hướng_dẫn th...


In [82]:
data = data.dropna()

In [83]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="wonrax/phobert-base-vietnamese-sentiment", device='cuda')

In [84]:
def map_label(label):
    if isinstance(label, list):
        return [map_label(l) for l in label]
    if label == 'POS':
        return 2
    elif label == 'NEU':
        return 1
    else:
        return 0

In [85]:
gt = data['sentiment'].tolist()
topic = data['topic'].tolist()
topicname = np.unique(topic)

In [86]:
print([v for v in data['sentence_dropped'] if not isinstance(v, str)])

[]


In [87]:
output = pipe(data['sentence_dropped'].tolist())

In [88]:
pred = [map_label(pred['label']) for pred in output]

In [89]:
# Split by topic
group_pred = [[] for i in topicname]
group_gt = [[] for i in topicname]
for p, g, t in zip(pred, gt, topic):
    group_pred[t].append(p)
    group_gt[t].append(g)

In [90]:
# Calculate accuracy, presicion, recall, f1 using sklearn
from sklearn.metrics import classification_report
[print(f"Topic {t}: \n{classification_report(group_gt[t], group_pred[t])}") for t in topicname]

Topic 0: 
              precision    recall  f1-score   support

           0       0.91      0.30      0.45       791
           1       0.07      0.72      0.13        71
           2       0.88      0.79      0.83      1424

    accuracy                           0.62      2286
   macro avg       0.62      0.60      0.47      2286
weighted avg       0.86      0.62      0.68      2286

Topic 1: 
              precision    recall  f1-score   support

           0       0.96      0.23      0.37       415
           1       0.12      0.83      0.20        47
           2       0.46      0.56      0.51       108

    accuracy                           0.34       570
   macro avg       0.51      0.54      0.36       570
weighted avg       0.80      0.34      0.38       570

Topic 2: 
              precision    recall  f1-score   support

           0       0.99      0.49      0.66       138
           1       0.04      1.00      0.07         2
           2       0.10      0.40      0.15  

[None, None, None, None]